In [1]:
#Graphlab

import graphlab as glb
from graphlab import SFrame
from graphlab import SArray

#numpy,pandas，preprocessing

import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler

In [2]:
#导入训练特征

all_feature_train=pd.read_csv('all_feature_input/all_features_train.csv')

#导入测试集特征

all_feature_test=pd.read_csv('all_feature_input/all_features_test.csv')

In [2]:
#导入训练特征

#all_feature_train=pd.read_csv('all_feature_input/all_features_train_2.csv')

#导入测试集特征

#all_feature_test=pd.read_csv('all_feature_input/all_features_test_2.csv')

In [10]:
all_feature_test.head()

,id,money,POS_recharge_mean,amount_mean,bus_mean,canteen_mean,card_recharge_mean,f_ten_canteen_mean,f_ten_place_mean,hospital_mean,...,borrow_resting_day_rate,borrow_working_day_rate,borrow_day_sum,borrow_day_per,0_percent,1000_percent,1500_percent,2000_percent,amount_college_rank,amount_score_rank
0,316,0.0,0.970361,6.086440,0.046161,0.291770,0.022220,0.174279,0.328624,-0.000149,...,0.0,0.0,0.0,0.0,0.353063,0.021807,0.020768,0.010384,213.0,160176.0
1,7399,0.0,0.970488,8.164384,0.063330,0.353499,0.031007,0.151998,0.310721,0.000158,...,0.0,0.0,0.0,0.0,0.353063,0.021807,0.020768,0.010384,170.0,70720.0
2,12370,0.0,0.944889,8.451108,0.110569,0.357973,0.023152,0.120780,0.277767,-0.000161,...,0.0,0.0,0.0,0.0,0.353063,0.021807,0.020768,0.010384,58.0,37584.0
3,21479,0.0,0.947016,8.281770,0.082200,0.276062,0.009322,0.170226,0.321603,0.000231,...,0.0,1.0,1.0,1.0,0.353063,0.021807,0.020768,0.010384,128.0,88320.0
4,29078,0.0,0.945593,7.880259,0.067525,0.282774,0.025579,0.108638,0.167391,0.000134,...,0.0,0.0,0.0,0.0,0.353063,0.021807,0.020768,0.010384,78.0,50622.0


In [27]:
len(all_feature_train)

10885

In [6]:
label='money'
IDcol='id'
ids=all_feature_test['id'].values

In [4]:
Strain=SFrame(all_feature_train)
Strain=Strain.remove_column('id')
all_feature_train['money']=all_feature_test['money'].apply(lambda x:int(x))
predictor=[x for x in all_feature_train.columns if x not in ['id','money']]

In [15]:
a={'0':1,'2000':2}
a

{'0': 1, '2000': 2}

In [16]:
#模型
model_1 =glb.boosted_trees_classifier.create(Strain, target=label,features=predictor,random_seed=10)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: Detected extremely low variance for feature(s) 'lib_hour_other_per_day', 'dorm_hour00_day_mean', 'dorm_hour01_day_mean', 'dorm_hour02_day_mean', 'dorm_hour03_day_mean', 'dorm_hour04_day_mean', 'dorm_hour05_day_mean', 'dorm_hour09_day_mean', 'dorm_hour10_day_mean', 'dorm_hour13_day_mean', 'dorm_hour14_day_mean', 'dorm_hour15_day_mean', 'dorm_hour16_day_mean', 'dorm_hour19_day_mean', 'dorm_hour20_day_mean', 'dorm_hour23_day_mean', 'dorm_hour_01_05_day_mean' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 10346

...

| 4         | 3.862570     | 0.885656          | 0.866419            | 0.545296          | 0.625696            |

| 5         | 4.824487     | 0.886913          | 0.866419            | 0.474920          | 0.574974            |

| 6         | 5.915766     | 0.889812          | 0.866419            | 0.421783          | 0.538511            |

| 10        | 9.460864     | 0.905954          | 0.864564            | 0.289618          | 0.486296            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [32]:
#需要找到合适的随机种子才能达到分层cv，本代码没有找到适合随机种子
    
sf=glb.toolkits.cross_validation.shuffle(Strain,random_seed=True)
folds = glb.cross_validation.KFold(sf, 5)
m = glb.boosted_trees_classifier.create(sf, target=label)
for train, valid in folds:
    m = glb.boosted_trees_classifier.create(sf, target=label,validation_set=valid)
    a=[1000,1500,2000]
    for i in range(len(a)):
        print len(valid[valid['money']==a[i]])
        print m.evaluate(valid)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: Detected extremely low variance for feature(s) 'lib_hour_other_per_day', 'dorm_hour00_day_mean', 'dorm_hour01_day_mean', 'dorm_hour02_day_mean', 'dorm_hour03_day_mean', 'dorm_hour04_day_mean', 'dorm_hour05_day_mean', 'dorm_hour09_day_mean', 'dorm_hour10_day_mean', 'dorm_hour13_day_mean', 'dorm_hour14_day_mean', 'dorm_hour15_day_mean', 'dorm_hour16_day_mean', 'dorm_hour19_day_mean', 'dorm_hour20_day_mean', 'dorm_hour23_day_mean', 'dorm_hour_01_05_day_mean' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 10349

...

| 8         | 8.750142     | 0.897198          | 0.888379            | 0.344869          | 0.356364            |

| 9         | 9.825457     | 0.901332          | 0.890216            | 0.315167          | 0.325919            |

| 10        | 10.838627    | 0.903721          | 0.891594            | 0.292946          | 0.303825            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

168
{'f1_score': 0.5705305311459775, 'auc': 0.9832007556696533, 'recall': 0.4671306843562941, 'precision': 0.9608043953699259, 'log_loss': 0.3038247584551167, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int
	class	int

Rows: 400004

Data:
+-----------+-----+-----+------+-----+-------+
| threshold | fpr | tpr |  p   |  n  | class |
+-----------+-----+-----+------+-----+-------+
|    0.0    | 1.0 | 1.0 | 1849 | 328 |   0   |
|   1e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   2e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   3e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   4e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   5e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   6e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   7e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   8e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   9e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
+-----------+-----+-----+------+-----+-------+
[400004 rows x 6 columns]
Note: Only the head of the SFrame is printed.
You 

In [33]:
feature_important=m.get_feature_importance()
feature_important.print_rows(num_rows=20)
#feature_important.tail()

+-------------------------------+-------+-------+
|              name             | index | count |
+-------------------------------+-------+-------+
|       POS_recharge_mean       |  None |   36  |
|            bus_mean           |  None |   32  |
|     hour_06_07_amount_sum     |  None |   30  |
|          score_order          |  None |   29  |
|   hour_06_07_amount_sum_rate  |  None |   26  |
|  amount_slice_day_10_20_count |  None |   21  |
|        hour_06_07_mean        |  None |   21  |
|         on_school_mean        |  None |   20  |
|          amount_mean          |  None |   19  |
|         remainder_max         |  None |   18  |
|          card_day_sum         |  None |   17  |
|      summer_vacation_mean     |  None |   16  |
|        hour_08_09_mean        |  None |   16  |
| card_recharge_amount_sum_rate |  None |   16  |
|       landry_amount_mean      |  None |   15  |
|        lib_morning_mean       |  None |   14  |
|          shower_mean          |  None |   14  |


In [39]:
feature_important[:500]['name']

dtype: str
Rows: 500
['POS_recharge_mean', 'bus_mean', 'hour_06_07_amount_sum', 'score_order', 'hour_06_07_amount_sum_rate', 'amount_slice_day_10_20_count', 'hour_06_07_mean', 'on_school_mean', 'amount_mean', 'remainder_max', 'card_day_sum', 'summer_vacation_mean', 'hour_08_09_mean', 'card_recharge_amount_sum_rate', 'landry_amount_mean', 'lib_morning_mean', 'shower_mean', 'shower_amount_std', 'f_ten_canteen_amount_std', 'load_tansfer_amount_sum_rate', 'canteen_mean', 'POS_recharge_amount_std', 'amount_slice_5_10_count_rate', 'hour_other_amount_sum', 'amount_slice_10_20_count_rate', 'shower_amount_mean', 'resting_day_mean', 'hour_11_13_mean', 'hour_17_21_mean', 'lib_hour_07', 'lib_hour_07_mean', 'hospital_mean', 'market_mean', 'card_recharge_mean', 'f_ten_place_amount_std', 'dorm_hour_afternoon_rate', 'POS_recharge_amount_sum_rate', 'f_ten_canteen_mean', 'hour_11_13_amount_mean', 'dorm_hour15_rate', 'hour_other_mean', 'amount_slice_2.5_5_rate_day', 'winter_vacation_mean', 'amount_slice_

In [34]:
m

Class                          : BoostedTreesClassifier

Schema
------
Number of examples             : 10885
Number of feature columns      : 584
Number of unpacked features    : 584
Number of classes              : 4

Settings
--------
Number of trees                : 40
Max tree depth                 : 6
Training time (sec)            : 10.8386
Training accuracy              : 0.9037
Validation accuracy            : 0.8916
Training log_loss              : 0.2929
Validation log_loss            : 0.3038

In [35]:
model_1

Class                          : BoostedTreesClassifier

Schema
------
Number of examples             : 10333
Number of feature columns      : 584
Number of unpacked features    : 584
Number of classes              : 4

Settings
--------
Number of trees                : 40
Max tree depth                 : 6
Training time (sec)            : 9.6048
Training accuracy              : 0.9075
Validation accuracy            : 0.8732
Training log_loss              : 0.2872
Validation log_loss            : 0.4487

In [40]:
feature_important1=model_1.get_feature_importance()
feature_important1.print_rows(num_rows=20)
#feature_important.tail()

+--------------------------------+-------+-------+
|              name              | index | count |
+--------------------------------+-------+-------+
|         on_school_mean         |  None |   31  |
|     hour_06_07_amount_sum      |  None |   31  |
|       POS_recharge_mean        |  None |   31  |
|          amount_mean           |  None |   27  |
|   hour_06_07_amount_sum_rate   |  None |   26  |
|        hour_06_07_mean         |  None |   23  |
|         remainder_max          |  None |   21  |
|          card_day_sum          |  None |   20  |
|            bus_mean            |  None |   20  |
|          score_order           |  None |   19  |
|  POS_recharge_amount_sum_rate  |  None |   18  |
|       card_recharge_mean       |  None |   17  |
| amount_slice_day_10_20day_rate |  None |   17  |
|  amount_slice_day_10_20_count  |  None |   17  |
|        resting_day_mean        |  None |   17  |
|        hour_08_09_mean         |  None |   16  |
|       shower_amount_std      

In [50]:
#模型
predictor=[x for x in Strain.column_names() if x not in feature_important[501:]['name']]
low_variance_feature=['dorm_hour00_day_mean', 'dorm_hour01_day_mean', 'dorm_hour02_day_mean', 
                      'dorm_hour03_day_mean', 'dorm_hour04_day_mean', 'dorm_hour05_day_mean', 
                      'dorm_hour14_day_mean', 'dorm_hour15_day_mean', 'dorm_hour16_day_mean', 
                      'dorm_hour19_day_mean', 'dorm_hour20_day_mean', 
                      'dorm_hour23_day_mean', 'dorm_hour_01_05_day_mean']
predictor_1=[x for x in predictor if x not in low_variance_feature]
model_2 =glb.boosted_trees_classifier.create(Strain[predictor_1], target=label)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 10378

Number of classes           : 4

...

| 4         | 3.545061     | 0.884178          | 0.852071            | 0.544734          | 0.638933            |

| 5         | 4.415969     | 0.887262          | 0.850099            | 0.473079          | 0.587794            |

| 6         | 5.298084     | 0.888803          | 0.850099            | 0.418958          | 0.551858            |

| 10        | 8.721715     | 0.906533          | 0.852071            | 0.287844          | 0.485440            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [51]:
#需要找到合适的随机种子才能达到分层cv，本代码没有找到适合随机种子
    
sf_1=glb.toolkits.cross_validation.shuffle(Strain[predictor_1],random_seed=True)
folds = glb.cross_validation.KFold(sf_1, 5)
m = glb.boosted_trees_classifier.create(sf_1, target=label)
for train, valid in folds:
    m = glb.boosted_trees_classifier.create(sf_1, target=label,validation_set=valid)
    a=[1000,1500,2000]
    for i in range(len(a)):
        print len(valid[valid['money']==a[i]])
        print m.evaluate(valid)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 10344

Number of classes           : 4

...

| 5         | 4.950133     | 0.886909          | 0.875976            | 0.474044          | 0.485838            |

| 6         | 5.955855     | 0.889389          | 0.877814            | 0.420495          | 0.433212            |

| 10        | 9.834127     | 0.903721          | 0.891594            | 0.292946          | 0.303825            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

168
{'f1_score': 0.5705305311459775, 'auc': 0.9832007556696533, 'recall': 0.4671306843562941, 'precision': 0.9608043953699259, 'log_loss': 0.3038247584551167, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int
	class	int

Rows: 400004

Data:
+-----------+-----+-----+------+-----+-------+
| threshold | fpr | tpr |  p   |  n  | class |
+-----------+-----+-----+------+-----+-------+
|    0.0    | 1.0 | 1.0 | 1849 | 328 |   0   |
|   1e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   2e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   3e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   4e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   5e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   6e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   7e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   8e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
|   9e-05   | 1.0 | 1.0 | 1849 | 328 |   0   |
+-----------+-----+-----+------+-----+-------+
[400004 rows x 6 columns]
Note: Only the head of the SFrame is printed.
You 

In [56]:
Stest=SFrame(all_feature_test)
predictions= m.classify(Stest[predictor_1])
predictions.show()

Canvas is accessible via web browser at the URL: http://localhost:4761/index.html
Opening Canvas in default web browser.


In [67]:
#predictions2=m.predict(Stest)
predictions3=model_2.predict(Stest)

In [68]:
test_result=SFrame()

In [69]:
test_result=test_result.add_columns([SArray(ids),predictions2],namelist=['studentid','subsidy'])

In [70]:
test_result

studentid,subsidy
316,0
7399,0
12370,0
21479,0
29078,0
87,0
106,0
107,0
144,0
189,0


In [72]:
len(test_result[test_result['subsidy']==1000])

44

In [33]:
test_result2=test_result.to_dataframe()

In [34]:
test_result2.to_csv('output/result_submit.csv',index=False)

In [35]:
test_result2.to_csv('output/result_submit_2.csv',index=False)